In [99]:
import asammdf
import hashlib

In [ ]:
import pony.orm

In [93]:
pony.orm.set_sql_debug(True)

In [114]:
db = pony.orm.Database()
db.bind(provider='sqlite', filename=':memory:')

class MDF(db.Entity):
    name = pony.orm.Required(
        str,
    )
    sha256 = pony.orm.Required(
        str, 
        unique=True,
    )
    md5 = pony.orm.Required(
        str,
        unique=True,
    )
    channels = pony.orm.Set(
        'Channel',
    )
    
class Channel(db.Entity):
    name = pony.orm.Required(
        str,
        unique=True,
    )
    mdfs = pony.orm.Set(
        "MDF",
    )

db.generate_mapping(create_tables=True)

GET NEW CONNECTION
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
PRAGMA foreign_keys = false
BEGIN IMMEDIATE TRANSACTION
CREATE TABLE "Channel" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT UNIQUE NOT NULL
)

CREATE TABLE "MDF" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT NOT NULL,
  "sha256" TEXT NOT NULL,
  "md5" TEXT NOT NULL
)

CREATE TABLE "Channel_MDF" (
  "channel" INTEGER NOT NULL REFERENCES "Channel" ("id"),
  "mdf" INTEGER NOT NULL REFERENCES "MDF" ("id"),
  PRIMARY KEY ("channel", "mdf")
)

CREATE INDEX "idx_channel_mdf" ON "Channel_MDF" ("mdf")

SELECT "Channel"."id", "Channel"."name"
FROM "Channel" "Channel"
WHERE 0 = 1

SELECT "Channel_MDF"."channel", "Channel_MDF"."mdf"
FROM "Channel_MDF" "Channel_MDF"
WHERE 0 = 1

SELECT "MDF"."id", "MDF"."name", "MDF"."sha256", "MDF"."md5"
FROM "MDF" "MDF"
WHERE 0 = 1

COMMIT
PRAGMA foreign_keys = true
CLOSE CONNECTION


In [115]:
data_file = "A1_f1_c2.mf4"

In [116]:
with open(data_file, "rb") as fid:
    sha256 = hashlib.sha256(fid.read()).hexdigest()
    md5 = hashlib.md5(fid.read()).hexdigest()

In [124]:
mdf = asammdf.MDF(data_file)

In [118]:
channels = list()
mdf.channels_db.keys()
for channel in mdf.channels_db.keys():
    channels.append(Channel(name=channel))
pony.orm.commit()

GET CONNECTION FROM THE LOCAL POOL
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Channel" ("name") VALUES (?)
['t']

INSERT INTO "Channel" ("name") VALUES (?)
['sawtooth']

INSERT INTO "Channel" ("name") VALUES (?)
['sine']

INSERT INTO "Channel" ("name") VALUES (?)
['square']

COMMIT


In [128]:
mdf_ = MDF(
    name=data_file,
    sha256=sha256,
    md5=md5,
    channels = channels,
)
pony.orm.commit()

BEGIN IMMEDIATE TRANSACTION
INSERT INTO "MDF" ("name", "sha256", "md5") VALUES (?, ?, ?)
['A1_f1_c2.mf4', '75e4293b5c6f626155de9f81c9d4ce95cfd367a78d6e96038ceb32c5e46cc856', 'd41d8cd98f00b204e9800998ecf8427e']

INSERT INTO "MDF" ("name", "sha256", "md5") VALUES (?, ?, ?)
['A1_f1_c2.mf4', '75e4293b5c6f626155de9f81c9d4ce95cfd367a78d6e96038ceb32c5e46cc856', 'd41d8cd98f00b204e9800998ecf8427e']

EXECUTEMANY (8)
INSERT INTO "Channel_MDF" ("channel", "mdf") VALUES (?, ?)
[3, 2]
[1, 1]
[2, 1]
[4, 2]
[2, 2]
[1, 2]
[3, 1]
[4, 1]

COMMIT


In [ ]:
db.